In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from spectroscopy.dataset import Dataset
from spectroscopy.visualize import plot
import tempfile
import numpy as np

## General structure of the datamodel

In [ ]:
from IPython.display import Image
Image(filename='../docs/_images/datamodel_simple.png')

## Load and explore an existing data file
In the following we load a dataset from one day of SO<sub>2</sub> monitoring using MiniDOAS sensors on White Island, New Zealand.

In [ ]:
d = Dataset.open('../tests/data/minidoas_test.h5')

In [ ]:
# Get the first PreferredFlux element
pf = d.elements['PreferredFlux'][0]
pf

In [ ]:
# Show the values of this element
pf.value[:]

In [ ]:
# Get the Flux element the PreferredFlux elements points to
f = pf.fluxes[0]

# Get the Flux value that corresponds to the second PreferredFlux value
idx = pf.flux_indices[0,1]
print f.value[idx]

# Get the Concentration element and the Concentration indices that correspond to 
# this particular Flux value
c = f.concentration
idx0, idx1 = f.concentration_indices[idx]

# Get the Rawdata element (i.e. the raw spectra) that correspond to Concentration values
r = c.rawdata[0]

# Plot the Concentration values for one scan
plt.figure()
plt.plot(r.inc_angle[c.rawdata_indices[idx0:idx1+1]], c.value[idx0:idx1+1])

In [ ]:
# Show an overview plot showing all scans for a day (in UTC).
# Every vertical line corresponds to one scan
plot(c, angle_bin=4.0)

In [ ]:
# Show an overview plot of all raw-spectra for 1 hour of measurements
plot(r, datemin='2016-10-31T22:00:00', datemax='2016-10-31T23:00:00')

In [ ]:
# Get the gasflow element for the flux estimate
# In this case this represents the wind speed and
# direction at the center of the plume
gf = f.gasflow

In [ ]:
plot(gf, vent=r.target.position[:])

In [ ]:
# Show instrument and target (i.e. vent) information
print repr(r.instrument)
print repr(r.target)

In [ ]:
# Show target position
r.target.position[:]

In [ ]:
# Close the dataset
d.close()

## Create a dataset from a Flyspec measurement file

In [ ]:
filename = "../tests/data/2012_02_29_1340_CHILE.txt"
d = Dataset(tempfile.mktemp(),'w')
e = d.read(filename, ftype='FLYSPEC', timeshift=-21)
rdt = d.new(e['RawDataTypeBuffer'])
rb = e['RawDataBuffer']
rb.type = rdt
r = d.new(rb)
cb = e['ConcentrationBuffer']
cb.rawdata = [r]
cb.rawdata_indices = np.arange(r.inc_angle.shape[0])
c = d.new(cb)
plot(c, timeshift=-21)